# Работа со строковыми значениями

Материалы:
* Макрушин С.В. Лекция 8: Работа со строковыми значениям
* https://pyformat.info/
* https://docs.python.org/3/library/re.html
* https://tproger.ru/translations/regular-expression-python/
* https://realpython.com/nltk-nlp-python/

## Задачи для совместного разбора

1. Вывести на экран данные из словаря `obj` построчно в виде `k = v`, задав формат таким образом, чтобы знак равенства оказался на одной и той же позиции во всех строках. Строковые литералы обернуть в кавычки.

In [1]:
obj = {  
         'home_page': 'https://github.com/pypa/sampleproject',
         'keywords': 'sample setuptools development',
         'license': 'MIT'
      }

2. Дана строка 'aaa--bbb==ccc__ddd'. Написать регулярное выражение для разбивки строки на список ['aaa','bbb','ccc','ddd'].

In [2]:
import re

In [3]:
s = 'aaa--bbb==ccc__ddd'
pattern = re.compile(r'[a-z]{3}')
pattern.findall(s)

['aaa', 'bbb', 'ccc', 'ddd']

3. Проверить корректность введенного E-mail

In [4]:
email = input('Enter e-mail>>> ')
pattern = re.compile(r'\w+@[a-z]+\.[a-z]{2,4}$')
pattern.match(email)

Enter e-mail>>> vk@ya.ru


<re.Match object; span=(0, 8), match='vk@ya.ru'>

4. Разбейте текст формулировки задачи 1 на слова.

In [5]:
import nltk

In [6]:
# nltk.download('punkt')

In [7]:
test = '1. Вывести на экран данные из словаря `obj` построчно в виде `k = v`, задав формат таким образом, чтобы знак равенства оказался на одной и той же позиции во всех строках. Строковые литералы обернуть в кавычки.'
nltk.word_tokenize(test)

['1',
 '.',
 'Вывести',
 'на',
 'экран',
 'данные',
 'из',
 'словаря',
 '`',
 'obj',
 '`',
 'построчно',
 'в',
 'виде',
 '`',
 'k',
 '=',
 'v',
 '`',
 ',',
 'задав',
 'формат',
 'таким',
 'образом',
 ',',
 'чтобы',
 'знак',
 'равенства',
 'оказался',
 'на',
 'одной',
 'и',
 'той',
 'же',
 'позиции',
 'во',
 'всех',
 'строках',
 '.',
 'Строковые',
 'литералы',
 'обернуть',
 'в',
 'кавычки',
 '.']

In [8]:
nltk.sent_tokenize(test)

['1.',
 'Вывести на экран данные из словаря `obj` построчно в виде `k = v`, задав формат таким образом, чтобы знак равенства оказался на одной и той же позиции во всех строках.',
 'Строковые литералы обернуть в кавычки.']

## Лабораторная работа 8

### Форматирование строк

1.1 Загрузите данные из файла `recipes_sample_with_tags_ingredients.csv` (__ЛР5__) в таблицу `recipes` При помощи форматирования строк выведите информацию об id рецепта и кол-ве игредиентов 5 случайных рецептов в виде таблицы следующего вида:

    
    |    id     | n_in  |
    |-------------------|
    |  400894   |  13   |
    |   68588   |   8   |
    |  362081   |   6   |
    |   53408   |  12   |
    |  221203   |   4   |

In [9]:
import pandas as pd

In [10]:
recipes = pd.read_csv('../sem05/result/recipes_sample_with_tags_ingredients.csv', sep=',', index_col=0)
recipes_sample = recipes.sample(n=5, random_state=1)

# строки подбивала на глаз, чтобы вывод был больше похож
print('|{r1:^11}|{r2:^7}|'.format(r1 = 'id', r2 = 'n_in'))
print('|{:-^11}|{:-^7}|'.format('-', '-'))

for index, row in recipes_sample.iterrows():
    print('|{r1:^11}|{r2:^7}|'.format(r1 = row[2], r2 = row[-2]))

|    id     | n_in  |
|-----------|-------|
|    160    |  15   |
|    65     |  12   |
|    70     |   8   |
|    35     |   7   |
|    106    |   8   |


1.2 Напишите функцию `show_info`, которая для рецепта по его `id` создает строку (в смысле объекта python) с описанием следующего вида:

```
"Название"

1. Шаг 1.
2. Шаг 2.
----------
#тэг1 #тэг2
```

    
Данные для создания строки получите из файлов `recipes_sample_with_tags_ingredients.csv`, `steps_sample.xml` (__ЛР4__) и `tags_sample.csv` (__ЛР5__). 
Выведите созданную строку на экран.

In [11]:
import requests
from bs4 import BeautifulSoup
import csv

In [12]:
def reading_data():
    recipe_steps_dics = {}
    tags_dict = {}
    
    with open('../sem04/data/steps_sample.xml', 'r', encoding='utf8') as bsf:
        ab = BeautifulSoup(bsf, 'xml')
        for recipe in ab.find_all('recipe'):
            for ids in recipe.find_all('id'):
                recipe_steps_dics[ids.text] = []
                for step in recipe.find_all('step'):
                    recipe_steps_dics[ids.text].append(step.text)
    
    with open('../sem05/data/tags_sample.csv') as fp:
        reader = csv.reader(fp)
        head = next(reader)
        for row in reader:
            if not int(row[0]) in tags_dict.keys():
                tags_dict[int(row[0])] = [row[1]]
            else:
                tags_dict[int(row[0])].append(row[1])
                
    recipes_with_steps = pd.read_csv('../sem05/result/recipes_sample_with_tags_ingredients.csv', sep=',', index_col=0)
    
    return recipe_steps_dics, tags_dict, recipes_with_steps 

In [13]:
def show_info(recipes_sample, id):
    
    recipe_steps_dics, tags_dict, recipes_with_steps = reading_data()
    
    row = recipes_sample[recipes_sample.id == id]
    res_string = f'"{row.iloc[0][1]}"\n\n'
    steps_lst = recipe_steps_dics[str(id)]
    
    for index, el in enumerate(steps_lst):
        res_string += f'{index+1}. {el.capitalize()}.\n'
        
    tags_lst = tags_dict[id]
    hashtags = ''
    for el in tags_lst:
        hashtags += f'#{el} '
        
    res_string += '{:-<{length}}\n'.format('-', length=114)
    res_string += hashtags
        
    return res_string

In [14]:
id = 78715 #рандомный, но существующий
result = show_info(recipes_sample, id)
print(result)

"78715"

1. Set oven to 300 degrees prepare two loaf pans , or 8 mini loaf pans.
2. In a large bowl combine the fruit , dates and walnuts , then dredge them with 1 / 4 cup flour.
3. Sift the remaining flour together with the baking powder , baking soda , salt , cinnamon , nutmeg and cloves.
4. Set aside.
5. In another bowl , cream butter , add sugar , and beat until the mixture is light and fluffy.
6. Beat in the eggs one at a time.
7. Beat in the orange rind , orange juice , orange extract and drained pineapple.
8. Mash the bananas with a fork.
9. Alternately , stir in the banana puree with the flour mixture into the creamed mixture.
10. Stir in the floured fruit and nuts.
11. Turn into prepared pans.
12. Bake for about 2 hours and 15 minutes , or until loosley with foil if the cakes are browning too much on top.
13. Cool in pans for 20 minutes , then turn onto racks.
------------------------------------------------------------------------------------------------------------------
#ti

## Работа с регулярными выражениями

В задачах данного блока подразумевается, что вы не будете использовать никаких строковые методы (`split`, `startswith` и т.д.). Все задачи необходимо решить при помощи регулярных выражений.

2.1 Посчитайте кол-во отзывов, в которых встречаются числа.

In [15]:
import re

In [16]:
reviews = pd.read_csv('../sem02/data/reviews_sample.csv', sep=',', index_col=0)

In [17]:
pattern = re.compile(r'[0-9]')
res = reviews.apply(
         lambda row: pattern.search(str(row.review)), axis=1)

In [18]:
# кол-во отзывов
res.dropna().shape[0]

49246

2.2 Найдите все смайлики в отзывах. Смайлик состоит из трех частей: глаза (символ `=` или `:`), нос (символ `-`), губы (символ `)` или `(`). Смайлик может иметь вид "глаза-нос-губы" или "губы-нос-глаза". Нос может отсутствовать.

In [19]:
pattern_22 = re.compile(r'[\(\)][-]?[:=]|[:=][-]?[\(\)]')
smiles_lst = []
for index, elem in reviews.iterrows():
    if pattern_22.findall(str(elem.review)):
        smiles_lst.extend(pattern_22.findall(str(elem.review)))
print(smiles_lst)

[':)', ':)', ':)', ':)', ':-)', ':)', ':)', ':)', ':)', ':)', ':)', ':)', ':-)', ':)', ':)', ':)', ':)', ':)', ':)', ':-)', ':)', ':)', ':)', ':)', ':)', ':)', ':)', ':)', '=)', ':)', ':)', ':)', ':)', ':)', ':)', ':)', ':)', ':)', ':)', ':)', ':)', ':)', ':)', ':)', ':)', ':-)', ':)', ':)', ':)', ':)', ':)', ':)', '):', ':)', '=)', ':)', ':-)', ':(', ':)', ':)', ':(', ':)', ':)', ':)', ':)', ':)', ':)', ':)', ':)', ':)', ':)', ':)', ':)', ':-)', ':)', ':)', ':)', ':)', ':)', ':)', ':)', ':-)', ':)', ':)', ':)', ':)', ':)', ':)', ':)', ':)', ':-)', ':)', ':)', ':)', ':)', '=)', ':-)', ':)', ':)', ':)', ':)', ':-)', ':-)', ':)', ':)', ':)', ':)', '=)', ':)', ':)', ':)', '=)', ':)', ':)', '=)', ':)', ':)', '=)', ':)', '):', '=)', ':-)', ':)', ':)', ':)', '=)', ':)', ':)', ':)', ':)', ':)', ':(', ':)', ':)', ':)', ':)', ':(', ':)', ':)', ':(', ':)', ':)', ':)', ':)', ':)', ':)', ':)', ':)', ':)', ':)', ':-(', ':)', ':)', ':)', '(:', ':)', ':)', ':)', ':)', ':)', ':)', ':)', ':)', ':)', ':

In [20]:
# уникальные
set(smiles_lst)

{'(-:', '(:', '(=', '):', ':(', ':)', ':-(', ':-)', '=(', '=)', '=-)'}

2.3 Проверьте, что все даты в датасете имеют вид "YYYY-MM-DD". Продемонстрируйте работу вашего решения, приведя пример из датасета и контрпример не из датасета.

*

я сделала для датасета из прошлого задания

In [21]:
reviews.shape[0]

126696

In [22]:
# по заданию необходимо провить только то, что вид ГГГГ-ММ-ДД, а не их достоверность (например, 9999-13-32)
pattern_23 = re.compile(r'^\d{4}-\d{2}-\d{2}$')
count = 0
for index, elem in reviews.iterrows():
    if pattern_22.findall(elem.date):
        count += 1
if count == reviews.shape[0]:
    print(f'все даты в датасете имеют вид "YYYY-MM-DD" ({count})')

In [23]:
# совпадение с данными из датасета
pattern_23.match(reviews.iloc[3]['date'])

<re.Match object; span=(0, 10), match='2017-12-11'>

In [24]:
# совпадение с данными не из датасета (вывод пустой - совпадения не найдено)
pattern_23.match("12-01-2013")

2.4 Используя строку-результат задачи 1.2, найдите первое слово каждого шага в рецепте

In [25]:
pattern_24 = re.compile(r'\n\d+. (\w+)')
for i in pattern_24.findall(result):
    print(i) 

Set
In
Sift
Set
In
Beat
Beat
Mash
Alternately
Stir
Turn
Bake
Cool


2.5 Используя регулярные выражения, удалите из описаний все символы, кроме английских букв, цифр и пробелов. Сохраните предобработанные описания в файл `preprocessed_descriptions.csv`, содержащий 2 столбца: `name` и `preprocessed_descriptions`.

In [26]:
recipes['preprocessed_description'] = recipes.apply(
         lambda row: re.sub(r"[^a-zA-Z1-9 ]+", "", str(row.description)),
         axis=1)

header = ["name", "preprocessed_description"]
recipes.to_csv('./result/preprocessed_descriptions.csv', columns = header)

In [27]:
pattern = re.compile(r'[0-9]')
res = recipes.apply(lambda row: pattern.search(str(row.preprocessed_description)), axis=1)

In [28]:
recipes['preprocessed_description'].iloc[4] 

'i think a fondue is a very romantic casual dinner or wonderful for after the theatre snack served with a robust red wine for dinner serve with rice  a small salad almond rice pilaf is a great accompaniment recipe posted separately  to cook the meat you must first heat your oil  i do this by heating it to almost boiling on the stove and then transfering it to your fondue burner buy good quality meat i recommend only using a fillet have at least 3 sauces if you want to serve 46 people just increase the meat to 2 lbs there will be enough saucethese sauce recipes came from '

### Сегментация текста

In [29]:
import nltk 

3.1 Разбейте отзывы из файла `recipes_sample.csv` (__ЛР2__) на предложения, а предложения - на слова (используйте `sent_tokenize` и `word_tokenize` из `nltk`). Каждый отзыв представьте в виде списка списков: внешний список - предложения, вложенные списки - слова в предложении. Исключите знаки препинания из списков слов.

`'Предложение номер один. Предложение номер два.' => [['Предложение', 'номер', 'один'], ['Предложение', 'номер', 'два']]`

In [30]:
# это решение чуть получше (по моему мнению), но здесь не используется word_tokenize
#   
# def list_pf_lists(row):
#     token = nltk.RegexpTokenizer(pattern = '[A-z]\w+')
#     sentences_list = nltk.sent_tokenize(row)
#     lst = []
#     for sentence in sentences_list:
#         lst.append(token.tokenize(str(sentence)))
#     return lst
#     

In [31]:
def list_pf_lists(row):
    sentences_list = nltk.sent_tokenize(row)
    lst = []
    for sentence in sentences_list:
        new = ' '.join(re.findall(r'[A-z]\w+', sentence))
        if new != '':
            lst.append(nltk.word_tokenize(new))
    return lst

*

возможно не из файла recipes, а из reviews т.к. в файле recipes_sample.csv нет отзывов

In [32]:
s = pd.read_csv('../sem02/data/reviews_sample.csv', sep=',', index_col=0)
reviews['review_list'] = reviews.apply(
         lambda row: list_pf_lists(str(row.review)),
         axis=1)

In [33]:
print(reviews.iloc[3].review_list)

[['These', 'are', 'favorite', 'for', 'the', 'holidays', 'and', 'so', 'easy', 'my', 'children', 'make', 'them'], ['We', 'also', 'use', 'Rolo', 'candies', 'in', 'place', 'of', 'the', 'kisses']]


In [34]:
print(f'{reviews.iloc[3].review} => {reviews.iloc[3].review_list}')

These are a favorite for the holidays and so easy my children make them. We also use Rolo candies in place of the kisses. => [['These', 'are', 'favorite', 'for', 'the', 'holidays', 'and', 'so', 'easy', 'my', 'children', 'make', 'them'], ['We', 'also', 'use', 'Rolo', 'candies', 'in', 'place', 'of', 'the', 'kisses']]


3.2 Посчитайте кол-во уникальных слов среди всех отзывов (без учета регистра и знаков препинания).

In [35]:
def list_of_words(row):
    token = nltk.RegexpTokenizer(pattern = '[A-z]\w+')
    sentences_list = token.tokenize(row.lower())
    return sentences_list
    

In [36]:
reviews['clean_review'] = reviews.apply(
         lambda row: list_of_words(str(row.review).lower()),
         axis=1)
reviews['FreqDist'] = reviews.apply(
         lambda row: nltk.FreqDist(row.clean_review),
         axis=1)
dicti = {}
unique = reviews.apply(
         lambda row: dicti.update(row.FreqDist),
         axis=1)

In [37]:
len(dicti.keys())

41810

In [38]:
# reviews.iloc[2].FreqDist.keys()

3.3 Найдите 5 самых длинных (по количеству слов; без учета знаков препинания) отзывов в датасете и выведите их в порядке убывания длины.

In [39]:
reviews['len'] = reviews.apply(
         lambda row: len(row.clean_review),
         axis=1)

nlargest = reviews.nlargest(5, 'len')

nl = nlargest.apply(
         lambda row: print(f'{row.review} \n\n *** \n\n'),
         axis=1)

One of my sisters was in need of some comfort food, and did a search for this to avoid having to dig through my Mother's heirloom stash of recipes in search of her original, grease stained label that she peeled off of the can of Crisco! lol! My only &quot;tweaks&quot; have been to add some dill, a tablespoon of lemon/lime juice, 1/2 t. of cayenne, and I often use dried onions, they will help the patty hold it's shape well and absorb any excess moistness, and I like to lightly press each patty into a dish of panko crumbs for extra crispness, especially if &quot;oven-frying&quot;. If I choose to pan fry, (which isn't often) I pass on using shortening (sorry, Crisco!) and use a healthier oil. My fav as of late is unrefined coconut oil! It gives a great crisp and delicious flavor to everything, and has a higher smoke point that olive oil. If you do fry, skip using canola. Although it is a &quot;healthier&quot; oil, it doesn't &quot;sear&quot; well when used to fry, pan fry, etc. and your f

In [40]:
reviews.nlargest(5, 'len').review

873900     One of my sisters was in need of some comfort ...
1031544    I am in the process of prepping all of my ingr...
419558     Your vet has severly mislead you and this diet...
1012304    I made these for a Valentine's day dinner and ...
1110888    I made these, and was worried that they would ...
Name: review, dtype: object

In [41]:
reviews.nlargest(5, 'len').len

873900     893
1031544    656
419558     642
1012304    627
1110888    622
Name: len, dtype: int64

3.4 Напишите функцию, которая для заданного предложения выводит информацию о частях речи слов, входящих в предложение, в следующем виде:
```
PRP   VBD   DT      NNS     CC   VBD      NNS        RB   
 I  omitted the raspberries and added strawberries instead
``` 
Для определения части речи слова можно воспользоваться `nltk.pos_tag`.

Проверьте работоспособность функции на любом предложении из отзывов.


In [42]:
# nltk.download('averaged_perceptron_tagger')

In [43]:
def pos_per_sentence_words(sentence):
    
    token = nltk.RegexpTokenizer(pattern = '[A-z]\w+')
    words_list = token.tokenize(sentence)
    sentence_tag = nltk.pos_tag(words_list)
    
    parts_of_speech, words = [], []
    
    for word_tuple in sentence_tag:
        word_len = max(map(len,word_tuple))
        word_lst = list(word_tuple)
        words.append("{0:^{word_len}}".format(word_lst[0], word_len=word_len))
        parts_of_speech.append("{0:^{word_len}}".format(word_lst[1],word_len=word_len))


    print(' '.join(parts_of_speech))
    print(' '.join(words))

In [44]:
print(nltk.sent_tokenize(reviews.iloc[0].review)[-1])

Super yummy, and leftovers for lunch today (lucky me)!


In [45]:
pos_per_sentence_words(nltk.sent_tokenize(reviews.iloc[0].review)[-1])

 NNP   NN   CC     NNS    IN   NN    NN    IN   PRP
Super yummy and leftovers for lunch today lucky me 


#### [версия 2]
* Уточнены формулировки задач 3.1-3.3